思路：先将全部个体的b0文件标准化得到每个个体配准到标准空间的转换矩阵trans_average.mat，再将转换矩阵用在每个个体个体空间的FA图像上得到每个个体标准空间的FA，并制作一个群体平均FA图像
在群体空间手动勾画ROI，再映射回每个个体空间

修改文件名，提取b0

In [ ]:
import os
import subprocess
image_data = '/Volumes/med_image/DTI_wang'
all_groups = os.listdir(image_data)
for group in all_groups:
    group_path = os.path.join(image_data, group)
    all_subjects = os.listdir(group_path)
    count = 0
    for subject in all_subjects:
        subject_path = os.path.join(group_path, subject)
        all_files = os.listdir(subject_path)
        for file in all_files:
            if file.endswith(('.nii', '.nii.gz')):
                try:
                    old_file_path = os.path.join(subject_path, file)
                    new_file_path = os.path.join(subject_path, 'DTI.nii.gz')
                    # 重命名DTI文件
                    os.rename(old_file_path, new_file_path)
                    output_b0_path = os.path.join(subject_path, 'b0.nii.gz')
                    subprocess.run(['fslroi', new_file_path, output_b0_path, '0', '1'], check=True)
                    count += 1
                    print(f"修改文件名，提取b0成功：{subject}, count={count}")
                except subprocess.CalledProcessError as e:
                    print(f"失败-{subject}：{e}")
print("批处理完成")

刚体变换，求平均

In [ ]:
import os
import subprocess
import  nibabel as nib
import numpy as np
image_data = '/Volumes/med_image/ALPS_wang/DTI_wang'
all_groups = os.listdir(image_data)
sum_image_data = None
count = 0
for group in all_groups:
    group_path = os.path.join(image_data, group)
    all_subjects = os.listdir(group_path)
    for subject in all_subjects:
        subject_path = os.path.join(group_path, subject)
        all_files = os.listdir(subject_path)
        for file in all_files:
            if file == "b0.nii.gz":
                try:
                    b0_path = os.path.join(subject_path, file)
                    flirt_out_path = os.path.join(subject_path,'b0_flirt.nii.gz')
                    mat_out_path = os.path.join(subject_path,'trans.mat')
                    subprocess.run(['flirt', '-in', b0_path, '-ref', '/Volumes/med_image/ALPS_wang/b0_renqiuyu.nii.gz', '-out', flirt_out_path, '-omat', mat_out_path, '-interp', 'spline', '-dof', '12'], check=True)
                    img = nib.load(flirt_out_path)
                    img_data = img.get_fdata()
                    if sum_image_data is None:
                        sum_image_data = np.zeros_like(img_data)
                    sum_image_data += img_data
                    count += 1
                    print(f"线性变换、累加完成：{subject}, count = {count}")
                except Exception as e:
                    print(f"失败e：{e}")
                except subprocess as r:
                    print(f"失败s：{r}")
    print(f"{group}处理完成")
print("批处理完成")

# 计算平均图像
average_image_data = sum_image_data / count

# 创建新NIfTI
average_img = nib.Nifti1Image(average_image_data, img.affine, img.header)
# 保存图像
nib.save(average_img,"/Volumes/med_image/ALPS_wang/average_wang/average_R.nii.gz")
print("完成")      

In [ ]:
import os
import subprocess
image_data = '/Volumes/med_image/ALPS_wang/DTI_wang'
all_groups = os.listdir(image_data)
for group in all_groups:
    group_path = os.path.join(image_data, group)
    all_subjects = os.listdir(group_path)
    count = 0
    for subject in all_subjects:
        subject_path = os.path.join(group_path, subject)
        all_files = os.listdir(subject_path)
        for file in all_files:
            if file == 'b0.nii.gz':
                try:
                    b0_path = os.path.join(subject_path, file)
                    flirt_average_out_path = os.path.join(subject_path,'b0_flirt_average.nii.gz')
                    mat_average_out_path = os.path.join(subject_path,'trans_average.mat')
                    subprocess.run(['flirt', '-in', b0_path, '-ref', '/Volumes/med_image/ALPS_wang/average_wang/average_R.nii.gz', '-out', flirt_average_out_path, '-omat', mat_average_out_path, '-interp', 'spline', '-dof', '12'], check=True)
                
                    count += 1
                    print(f"与b0平均图像配准完成：{subject}, count = {count}")
                
                except subprocess.CalledProcessError as r:
                    print(f"失败s：{r}")
                except Exception as e:
                    print(f"失败e：{e}")
    print(f"{group}处理完成")
print("批处理完成")
                

In [ ]:
import os
import subprocess

input_data_path = '/Volumes/med_image/ALPS_wang/result_wang'
all_groups = os.listdir(input_data_path)

for group in all_groups:
    group_path = os.path.join(input_data_path, group)
    all_subjects = os.listdir(group_path)
    count = 0
    for subject in all_subjects:
        subject_path = os.path.join(group_path, subject)
        image_path = os.path.join(subject_path, 'dti_results_FA.nii.gz')
        transform_matrix_path = os.path.join(subject_path, 'trans_average.mat')
        output_path = os.path.join(subject_path, 'FA_registered.nii.gz')
        flirt_commend = [
            'flirt',
            '-in', image_path,
            '-ref', image_path,
            '-out', output_path,
            '-applyxfm',
            '-init', transform_matrix_path
            
        ]

        
        try:
           
            
            subprocess.run(flirt_commend, check=True)
            
            count += 1
            print(f"{subject}处理完毕，count = {count}")
        except subprocess.CalledProcessError as e:
            print(f"处理失败：{e}")

In [ ]:
得到群体平均FA图像

In [ ]:
import  nibabel as nib
import numpy as np
image_data = '/Volumes/med_image/ALPS_wang/result_wang'
all_groups = os.listdir(image_data)
sum_image_data = None
count = 0
for group in all_groups:
    group_path = os.path.join(image_data, group)
    all_subjects = os.listdir(group_path)
    for subject in all_subjects:
        subject_path = os.path.join(group_path, subject)
        all_files = os.listdir(subject_path)
        for file in all_files:
            if file == "FA_registered.nii.gz":
                try:
                    Fa_path = os.path.join(subject_path, file)
                    
                    img = nib.load(Fa_path)
                    img_data = img.get_fdata()
                    if sum_image_data is None:
                        sum_image_data = np.zeros_like(img_data)
                    sum_image_data += img_data
                    count += 1
                    print(f"累加完成：{subject}, count = {count}")
                except Exception as e:
                    print(f"失败e：{e}")
                except subprocess as r:
                    print(f"失败s：{r}")
    print(f"{group}处理完成")
print("批处理完成")

# 计算平均图像
average_image_data = sum_image_data / count

# 创建新NIfTI
average_img = nib.Nifti1Image(average_image_data, img.affine, img.header)
# 保存图像
nib.save(average_img,"/Volumes/med_image/ALPS_wang/average_wang/average_FA.nii.gz")
print("完成")               
                

求出每个个体FA到FA_Template的转换矩阵

In [ ]:
import os
import subprocess
input_data_path = '/Volumes/med_image/ALPS_Wang/result_wang'
all_groups = os.listdir(input_data_path)

for group in all_groups:
    group_path = os.path.join(input_data_path, group)
    all_subjects = os.listdir(group_path)
    count = 0
    for subject in all_subjects:
        subject_path = os.path.join(group_path, subject)
        fa_path = os.path.join(subject_path, 'dti_results_FA.nii.gz')
        average_fa_path = '/Volumes/med_image/ALPS_Wang/average_wang/average_FA.nii.gz'
        out_mat_path = os.path.join(subject_path, 'FA2average.mat')
        flirt_out_path = os.path.join(subject_path, 'FA2average.nii.gz')
        inv_out_mat_path = os.path.join(subject_path, 'inverse_FA2average.mat')
        try:
            subprocess.run(['flirt', '-in', fa_path, '-ref', '/Volumes/med_image/ALPS_Wang/average_wang/average_FA.nii.gz', '-out', flirt_out_path, '-omat', out_mat_path, '-interp', 'spline', '-dof', '12'], check=True)
            subprocess.run(['convert_xfm', '-inverse', '-omat', inv_out_mat_path, out_mat_path], check=True)

            count += 1
            print(f"{subject}处理完毕，count = {count}")

        except subprocess.CalledProcessError as e:
            print(f"{subject}处理失败：{e}")

print("批处理完毕")

将变换矩阵应用到标准ROI，自动得到个体ROI

In [ ]:
import os
import subprocess

input_data_path = '/Volumes/med_image/ALPS_Wang/result_wang'
all_groups = os.listdir(input_data_path)

for group in all_groups:
    group_path = os.path.join(input_data_path, group)
    all_subjects = os.listdir(group_path)
    count = 0
    for subject in all_subjects:
        subject_path = os.path.join(group_path, subject)
        fa_path = os.path.join(subject_path, 'dti_results_FA.nii.gz')
        average_fa_path = '/Volumes/med_image/ALPS_Wang/average_wang/average_FA.nii.gz'
        out_mat_path = os.path.join(subject_path, 'FA2average.mat')
        flirt_out_path = os.path.join(subject_path, 'FA2average.nii.gz')
        inv_out_mat_path = os.path.join(subject_path, 'inverse_FA2average.mat')
        inv_flirt_out_path_1 = os.path.join(subject_path, 'l_asso_average.nii.gz')
        inv_flirt_out_path_2 = os.path.join(subject_path, 'l_proj_average.nii.gz')
        inv_flirt_out_path_3 = os.path.join(subject_path, 'r_asso_average.nii.gz')
        inv_flirt_out_path_4 = os.path.join(subject_path, 'r_proj_average.nii.gz')

        try:
            

            # 应用逆矩阵将average_FA图像变换回每个个体的FA空间
            subprocess.run(['flirt', '-in','/Volumes/med_image/ALPS_Wang/average_wang/l_asso.nii.gz', '-ref', fa_path, '-out', inv_flirt_out_path_1, '-init', inv_out_mat_path, '-applyxfm', '-interp', 'spline'], check=True)
            subprocess.run(['flirt', '-in','/Volumes/med_image/ALPS_Wang/average_wang/l_proj.nii.gz', '-ref', fa_path, '-out', inv_flirt_out_path_2, '-init', inv_out_mat_path, '-applyxfm', '-interp', 'spline'], check=True)
            subprocess.run(['flirt', '-in','/Volumes/med_image/ALPS_Wang/average_wang/r_asso.nii.gz', '-ref', fa_path, '-out', inv_flirt_out_path_3, '-init', inv_out_mat_path, '-applyxfm', '-interp', 'spline'], check=True)
            subprocess.run(['flirt', '-in','/Volumes/med_image/ALPS_Wang/average_wang/r_proj.nii.gz', '-ref', fa_path, '-out', inv_flirt_out_path_4, '-init', inv_out_mat_path, '-applyxfm', '-interp', 'spline'], check=True)

            count += 1
            print(f"{subject}处理完毕，count = {count}")

        except subprocess.CalledProcessError as e:
            print(f"{subject}处理失败：{e}")

print("批处理完毕")